# Predicting Malicious Cyber Connections
<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
</p>

The general setup for the problem is a common one: we have a single table of log lines recording Internet traffic between various sources. Traffic between a source and destination is labeled as malicious or clean in the dataset, and we'd like to be able to predict ahead of time if a future connection between a source and a destination will be malicious.

We'll demonstrate an end-to-end workflow using this [Cybersecurity Dataset](). This notebook demonstrates a rapid way to predict whether a connection (defined as a source name/destination name pair) is malicious.


## Highlights
* Quickly make end-to-end workflow using log-line cybersecurity data
* Find interesting automatically generated features

Note: this is an extremely imbalanced dataset, and would benefit tremendously from additional positive (malicious) labels

In [1]:
import featuretools as ft
from featuretools.primitives import CumMean, Percentile
from featuretools.selection import remove_low_information_features
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import utils

# Step 1: Understanding the Data
Here we load in the data and do a bit of preprocessing

In [2]:
cyber_df = pd.read_csv("data/CyberFLTenDays.csv")
cyber_df.index.name = "log_id"
cyber_df.reset_index(inplace=True, drop=False)
cyber_df['label'] = cyber_df['label'].map({'N': False, 'A': True}, na_action='ignore')

# Sample down negative examples because very few positives
# Can also do this after the feature engineering step (but doing it here reduces computation time)
cyber_df_pos = cyber_df[cyber_df['label']]
cyber_df_neg = cyber_df[~cyber_df['label']].sample(100000)
cyber_df = pd.concat([cyber_df_pos, cyber_df_neg]).sort_values(['secs'])

In [3]:
cyber_df.head()

,log_id,secs,src_name,dest_name,src_host,dest_host,auth_type,login_type,stage,result,label
10,10,25,U175@DOM1,U175@DOM1,C1085,C1085,?,?,TGS,Success,False
12,12,33,U6@DOM1,U6@DOM1,C921,C921,Kerberos,Network,LogOn,Success,False
20,20,91,U1391@DOM1,U1391@DOM1,C612,C612,?,Network,LogOff,Success,False
22,22,128,C1954$@DOM1,C1954$@DOM1,C1954,C1954,?,?,TGT,Success,False
28,28,151,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C659,C586,NTLM,Network,LogOn,Success,False


## Create an EntitySet
To apply Deep Feature Synthesis we need to establish an `EntitySet` structure for our data. Since we're interested in predicting for combinations of "src_name" and "dest_name" (we call this pair a "session"), we need to create a separate normalized entity for "sessions".

In [4]:
es = ft.EntitySet("CyberLL")

# create an index column
cyber_df["name_host_pair"] = cyber_df["src_name"].str.cat(
                                [cyber_df["dest_name"],
                                 cyber_df["src_host"],
                                 cyber_df["dest_host"]],
                                sep=' / ')
cyber_df["session_id"] = cyber_df["src_name"].str.cat(
                                 cyber_df["dest_name"],
                                 sep=' / ')

es.entity_from_dataframe("log",
                         cyber_df,
                         index="log_id",
                         time_index="secs")
es.normalize_entity(base_entity_id="log",
                    new_entity_id="name_host_pairs",
                    index="name_host_pair",
                    additional_variables=["src_name", "dest_name",
                                          "src_host", "dest_host",
                                          #"src_pair",
                                          #"dest_pair",
                                          "session_id",
                                          "label"])
es.normalize_entity(base_entity_id="name_host_pairs",
                    new_entity_id="sessions",
                    index="session_id",
                    additional_variables=["dest_name", "src_name"])

Entityset: CyberLL
  Entities:
    sessions [Rows: 19251, Columns: 4]
    log [Rows: 100329, Columns: 7]
    name_host_pairs [Rows: 62100, Columns: 6]
  Relationships:
    log.name_host_pair -> name_host_pairs.name_host_pair
    name_host_pairs.session_id -> sessions.session_id

In [5]:
cyber_df.head()

,log_id,secs,src_name,dest_name,src_host,dest_host,auth_type,login_type,stage,result,label,name_host_pair,session_id
10,10,25,U175@DOM1,U175@DOM1,C1085,C1085,?,?,TGS,Success,False,U175@DOM1 / U175@DOM1 / C1085 / C1085,U175@DOM1 / U175@DOM1
12,12,33,U6@DOM1,U6@DOM1,C921,C921,Kerberos,Network,LogOn,Success,False,U6@DOM1 / U6@DOM1 / C921 / C921,U6@DOM1 / U6@DOM1
20,20,91,U1391@DOM1,U1391@DOM1,C612,C612,?,Network,LogOff,Success,False,U1391@DOM1 / U1391@DOM1 / C612 / C612,U1391@DOM1 / U1391@DOM1
22,22,128,C1954$@DOM1,C1954$@DOM1,C1954,C1954,?,?,TGT,Success,False,C1954$@DOM1 / C1954$@DOM1 / C1954 / C1954,C1954$@DOM1 / C1954$@DOM1
28,28,151,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C659,C586,NTLM,Network,LogOn,Success,False,ANONYMOUS LOGON@C586 / ANONYMOUS LOGON@C586 / ...,ANONYMOUS LOGON@C586 / ANONYMOUS LOGON@C586


# Generate labels and associated cutoff times

Featuretools can generate features for each session strictly before an associated cutoff time. We find these cutoff times in the process of computing labels. Labels are defined as follows:

For a given session:
 * After seeing the same name/host pair N times
 * Predict L observations of this same session in the future
 * Where any connections from this session in a window of size W are malicious
 
We will set N = 2 (number of observations to wait for), L = 2 (lead time), and W = 10 (prediction window)

In [6]:
def generate_cutoffs(cyber_df, index_col, after_n_obs, lead, prediction_window):
    window_start = after_n_obs + lead
    window_end = window_start + prediction_window
    grouped = cyber_df.groupby(index_col)[index_col].count()
    grouped.name = "count"
    min_obs = after_n_obs + lead + 1
    enough_examples = grouped[grouped > min_obs].to_frame().reset_index()
    enough_examples = cyber_df[cyber_df[index_col].isin(enough_examples[index_col])]
    def get_label_and_cutoff(df):
        cutoff = df.iloc[after_n_obs]
        cutoff['label'] = df.iloc[window_start: window_end]["label"].any()
        return cutoff
    cutoffs = enough_examples.groupby(index_col)[[index_col, "secs", "label"]].apply(get_label_and_cutoff)
    return cutoffs

In [7]:
cutoffs = generate_cutoffs(cyber_df, "session_id", 2, 2, 10)

In [8]:
cutoffs['label'].value_counts()

False    4137
True       37
Name: label, dtype: int64

# Compute features using DFS

In [9]:
fm, fl = ft.dfs(entityset=es, target_entity="sessions", cutoff_time=cutoffs,
                cutoff_time_in_index=True,
                verbose=True, max_depth=3)

Built 98 features
Elapsed: 26:19 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [10]:
fm.head()

,,first_name_host_pairs_time,src_name,dest_name,NUM_UNIQUE(log.result),NUM_UNIQUE(name_host_pairs.src_host),MODE(log.stage),STD(name_host_pairs.first_log_time),MEAN(name_host_pairs.first_log_time),MODE(log.login_type),MIN(name_host_pairs.first_log_time),...,SKEW(name_host_pairs.NUM_UNIQUE(log.auth_type)),STD(name_host_pairs.MEAN(log.secs)),SKEW(name_host_pairs.MAX(log.secs)),SUM(name_host_pairs.NUM_UNIQUE(log.auth_type)),SKEW(name_host_pairs.COUNT(log)),MEAN(name_host_pairs.MAX(log.secs)),MIN(name_host_pairs.SKEW(log.secs)),STD(name_host_pairs.SUM(log.secs)),NUM_UNIQUE(name_host_pairs.MODE(log.result)),label
session_id,time,,,,,,,,,,,,,,,,,,,,,
ANONYMOUS LOGON@C586 / ANONYMOUS LOGON@C586,637,151,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,1,3,LogOn,243.362555,386.333333,Network,151,...,0.0,243.362555,0.282402,3,0.0,386.333333,NaN,243.362555,1,False
U66@DOM1 / U66@DOM1,1579,773,U66@DOM1,U66@DOM1,1,3,LogOff,444.298323,1284.000000,Network,773,...,0.0,444.298323,-1.670649,3,0.0,1284.000000,NaN,444.298323,1,False
C523$@DOM1 / C523$@DOM1,1845,1154,C523$@DOM1,C523$@DOM1,1,2,LogOn,357.119494,1551.666667,Network,1154,...,0.0,357.119494,-1.202474,3,0.0,1551.666667,NaN,357.119494,1,False
C567$@DOM1 / C567$@DOM1,2073,1172,C567$@DOM1,C567$@DOM1,1,3,LogOff,514.516278,1479.000000,Network,1172,...,0.0,514.516278,1.729107,3,0.0,1479.000000,NaN,514.516278,1,False
U22@DOM1 / U22@DOM1,2268,355,U22@DOM1,U22@DOM1,1,3,LogOn,974.124735,1205.000000,Network,355,...,0.0,974.124735,0.936916,3,0.0,1205.000000,NaN,974.124735,1,False


### Sort indexes to line up cutoffs with feature matrix

In [11]:
fm = fm.reorder_levels(['time', 'session_id']).sort_index()
cutoffs = cutoffs.set_index('secs', append=True).reorder_levels(['secs', 'session_id']).sort_index()
fm['label'] = cutoffs['label'].values

### One-Hot-Encode categorical features and remove features with low information

In [12]:
fm_encoded, fl_encoded = ft.encode_features(fm, fl)
fm_encoded, fl_encoded = remove_low_information_features(fm_encoded, fl_encoded)

# Machine Learning

Now that we have a feature matrix and associated labels, we can build a standard machine learning pipeline with a RandomForestClassifier

First, split up the data into train and test sets

In [13]:
train, test = train_test_split(fm_encoded, test_size=0.2, shuffle=True)

In [14]:
X_train = train
y_train = X_train.pop('label')
X_test = test
y_test = X_test.pop('label')

### Construct the model

In [15]:
imputer = Imputer(missing_values='NaN', strategy="mean", axis=0)
scaler = StandardScaler()
clf = RandomForestClassifier(n_jobs=-1)
model = Pipeline([("imputer", imputer),
                  ("scaler", scaler),
                  ("rf", clf)])

/Users/lecrintech/projects/python/demos-featuretools/demo9/predict-malicious-cyber-connections/env2.7/lib/python2.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### Fit the model, then score it

In [16]:
model.fit(X_train, y_train)
    
probs = model.predict_proba(X_test)
score = roc_auc_score(y_test, probs[:,1])
print('ROC AUC Score: {:.2f}'.format(score))

/Users/lecrintech/projects/python/demos-featuretools/demo9/predict-malicious-cyber-connections/env2.7/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ROC AUC Score: 0.55


## View the most important features
according to the Random Forest

In [17]:
high_imp_feats = utils.feature_importances(X_train, clf, feats=10)

1: PERCENT_TRUE(name_host_pairs.label) [0.100]
2: SUM(name_host_pairs.first_log_time) [0.062]
3: STD(log.secs) [0.042]
4: MAX(name_host_pairs.MEAN(log.secs)) [0.041]
5: MAX(name_host_pairs.first_log_time) [0.039]
6: SUM(name_host_pairs.MIN(log.secs)) [0.038]
7: MEAN(name_host_pairs.MEAN(log.secs)) [0.038]
8: MIN(name_host_pairs.SUM(log.secs)) [0.037]
9: STD(name_host_pairs.MAX(log.secs)) [0.034]
10: MEAN(log.secs) [0.034]
-----



## Save output files

In [18]:
import os

try:
    os.mkdir("output")
except:
    pass

fm_encoded.to_csv('output/feature_matrix.csv')
cutoffs.to_csv('output/cutoffs.csv')

<p>
    <img src="https://www.featurelabs.com/wp-content/uploads/2017/12/logo.png" alt="Featuretools" />
</p>

Featuretools was created by the developers at [Feature Labs](https://www.featurelabs.com/). If building impactful data science pipelines is important to you or your business, please [get in touch](https://www.featurelabs.com/contact/).